In [1]:
import numpy as np
import tensorflow as tf
from tqdm import trange 
import logging
import os
import pandas as pd
%run "./utils.py"

params = {
    "max_query_words": 12,
    "max_passage_words": 50,
    "emb_dim": 50,
    "BATCH_SIZE": 500,
    "EPOCHS" : 200,  #Total number of epochs to run
    "BatchesPerEpoch": 1000,
    "num_classes": 2,
    "save_summary_steps": 100,
    "TEST_BATCH_SIZE": 1000,
    "SHUFFLE_BATCH_SIZE": 10000
}

C:\Users\sitandon\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def cnn_network(queryfeatures, passagefeatures,reuse = False):
    
    #queryfeatures = tf.placeholder(tf.float32,shape=(None,max_query_words * emb_dim))
    #passagefeatures = tf.placeholder(tf.float32,shape=(None,max_passage_words * emb_dim))
    
    #global max_passage_words,max_query_words,emb_dim,num_classes

    def conv2D(x,W,strides):
        return tf.nn.conv2d(x,W,strides=strides,padding='VALID',name="conv2D")

    def maxPooling(x,k):
        return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='VALID',name = "maxPool")

    def getWeight(shape):
        return tf.get_variable(name = "weight",shape = shape, initializer=tf.initializers.truncated_normal(stddev= 0.1))

    def getBias(shape):
        return tf.get_variable(name = "bias",shape = shape, initializer=tf.initializers.constant(0.1))

    with tf.variable_scope('query/convLayer1'):
        #------Filter: [filter_height, filter_width, in_channels, out_channels]
        with tf.variable_scope('shared',reuse=reuse):
            weight = getWeight([3,10,1,4])
            bias = getBias([4])
        #------Input: [batch, in_height, in_width, in_channels]
        #x_query = tf.reshape(queryfeatures,[-1,max_query_words,emb_dim,1], name="ReshapeInputOp")  #input: ?,12,50,1
        #------Stride [batch,height,width,channel]
        convQuery1 = tf.nn.relu(conv2D(queryfeatures,weight,[1,1,1,1]) + bias)
        convQuery1_max = maxPooling(convQuery1,k = 2)

    with tf.variable_scope('query/convLayer2'):
        with tf.variable_scope('shared',reuse=reuse):
            weight = getWeight([2,4,4,2])
            bias = getBias([2])
        convQuery2 = tf.nn.relu(conv2D(convQuery1_max,weight,[1,1,1,1]) + bias)
        convQuery2_max = maxPooling(convQuery2,k = 2)
        print(convQuery2_max)

    with tf.variable_scope('query/denseLayer3'):
        with tf.variable_scope('shared',reuse=reuse):
            weight = getWeight([2*8*2,10])
            bias = getBias([10])
        dense = tf.reshape(convQuery2_max,[-1,2*8*2])
        denseQuery3 = tf.nn.relu(tf.matmul(dense,weight) + bias)

    with tf.variable_scope('passage/convLayer1'):
        with tf.variable_scope('shared',reuse=reuse):
            weight = getWeight([5,10,1,4])
            bias = getBias([4])
        #x_passage = tf.reshape(passagefeatures,[-1,max_passage_words,emb_dim,1])
        convPassage1 = tf.nn.relu(conv2D(passagefeatures,weight,[1,1,1,1]) + bias)
        convPassage1_max = maxPooling(convPassage1,k = 2)


    with tf.variable_scope('passage/convLayer2'):
        with tf.variable_scope('shared',reuse=reuse):
            weight = getWeight([3,10,4,4])
            bias = getBias([4])
        convPassage2 = tf.nn.relu(conv2D(convPassage1_max,weight,[1,1,1,1]) + bias)
        convPassage2_max = maxPooling(convPassage2,k = 2)

    with tf.variable_scope('passage/denseLayer3'):
        with tf.variable_scope('shared',reuse=reuse):
            weight = getWeight([10*5*4,10])
            bias = getBias([10])
        densePassage = tf.reshape(convPassage2_max,[-1,10*5*4])
        densePassage3 = tf.nn.relu(tf.matmul(densePassage,weight) + bias)

    with tf.variable_scope('mergeQueryPassage'):
        with tf.variable_scope('shared',reuse=reuse):
            weight = getWeight([10,2])
            bias = getBias([2])
        
        mergeOp = tf.multiply(denseQuery3,densePassage3, name = "merge")
        mergeDense = tf.nn.relu(tf.matmul(mergeOp,weight) + bias)

    return mergeDense
    

In [3]:
def modelTest_fn(mode,embeddingsFile,params):    
    #--------Hyper parameters:
    max_query_words = params["max_query_words"]
    max_passage_words = params["max_passage_words"]
    emb_dim = params["emb_dim"]
    BATCH_SIZE = params["TEST_BATCH_SIZE"]
    EPOCHS = params["EPOCHS"]
    BatchesPerEpoch = params["BatchesPerEpoch"]
    num_classes = params["num_classes"]
    
    is_training = (mode == "train")
    
    def testDSParser(example_proto):
        features = {"query": tf.FixedLenFeature((max_query_words,emb_dim,1), tf.float32),
                  "passage": tf.FixedLenFeature((max_passage_words,emb_dim,1), tf.float32),
                  "query_id": tf.FixedLenFeature((1), tf.int64),
                  "passage_id": tf.FixedLenFeature((1), tf.int64)}
        parsed_features = tf.parse_single_example(example_proto, features)
        return parsed_features["query"], parsed_features["passage"],parsed_features["query_id"],parsed_features["passage_id"]
    
    def getDatasetIterator(fileName,batch_size,mode):
        dataset = tf.data.TFRecordDataset(filenames = fileName, compression_type="ZLIB")
        #------Follow this order: map -> prefetch -> batch
        dataset = dataset.map(testDSParser)
        dataset = dataset.batch(batch_size)
        
        iterator = dataset.make_initializable_iterator()
        return iterator
    
    iterator = getDatasetIterator(embeddingsFile,BATCH_SIZE,mode)
    
    queryfeatures,passagefeatures,query_id,passage_id = iterator.get_next()
    
    model_spec =     {
        'queryfeatures': queryfeatures,
        'passagefeatures': passagefeatures,
        'iterator_init_op': iterator.initializer,
        "query_id":query_id,
        "passage_id": passage_id
    }
    
    with tf.variable_scope('model'):
        y_conv = cnn_network(queryfeatures,passagefeatures,reuse = False)
    
    # -----------------------------------------------------------
    # MODEL SPECIFICATION
    # Create the model specification and return it
    # It contains nodes or operations in the graph that will be used for training and evaluation
    variable_init_op = tf.group(*[tf.global_variables_initializer()])
    model_spec['variable_init_op'] = variable_init_op
    model_spec["predictions"] = y_conv
    model_spec['summary_op'] = tf.summary.merge_all()

    
    return model_spec

In [12]:
def model_fn(mode,embeddingsFile,params):    
    #--------Hyper parameters:
    max_query_words = params["max_query_words"]
    max_passage_words = params["max_passage_words"]
    emb_dim = params["emb_dim"]
    BATCH_SIZE = params["BATCH_SIZE"]
    EPOCHS = params["EPOCHS"]
    BatchesPerEpoch = params["BatchesPerEpoch"]
    num_classes = params["num_classes"]
    
    is_training = (mode == "train")
    
    def parser(example_proto):
        features = {"query": tf.FixedLenFeature((max_query_words,emb_dim,1), tf.float32),
                  "passage": tf.FixedLenFeature((max_passage_words,emb_dim,1), tf.float32),
                  "label": tf.FixedLenFeature((num_classes), tf.int64)}
        parsed_features = tf.parse_single_example(example_proto, features)
        return parsed_features["query"], parsed_features["passage"],parsed_features["label"]
    
    def getDatasetIterator(fileName,batch_size,mode):
        buffer_size = 4 * batch_size
        dataset = tf.data.TFRecordDataset(filenames = fileName, compression_type="ZLIB").shuffle(buffer_size = buffer_size)
        #------Follow this order: map -> prefetch -> batch
        dataset = dataset.map(parser)
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(1)
        iterator = dataset.make_initializable_iterator()
        return iterator
    
    #queryfeatures = tf.placeholder(tf.float32,shape=(None,max_query_words, emb_dim,1))
    #passagefeatures = tf.placeholder(tf.float32,shape=(None,max_passage_words,emb_dim,1))
    #y = tf.placeholder(tf.int16,shape = (None,num_classes))
    
    iterator = getDatasetIterator(embeddingsFile,BATCH_SIZE,mode)
    
    queryfeatures,passagefeatures,y = iterator.get_next()
    #print(queryfeatures)
    
    model_spec =     {
        'queryfeatures': queryfeatures,
        'passagefeatures': passagefeatures,
        'iterator_init_op': iterator.initializer,
        "y":y
    }
    
    with tf.variable_scope('model'):
        y_conv = cnn_network(queryfeatures,passagefeatures,reuse = not is_training)
    
    with tf.variable_scope('lossPerBatch'):
        cross_entropy = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels=y,logits=y_conv))
        tf.summary.scalar('lossPerBatch', cross_entropy)

    if is_training:
        with tf.name_scope('AdamOptim'):
            global_step = tf.train.get_or_create_global_step()
            train_step = tf.train.AdamOptimizer(1e-4).minimize(loss = cross_entropy,global_step=global_step)

    correct_pred = tf.equal(tf.argmax(y_conv,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
    tf.summary.scalar('accuracyPerBatch', accuracy)
    
    
    # -----------------------------------------------------------
    # METRICS AND SUMMARIES
    # Metrics for evaluation using tf.metrics (average over whole dataset)
    with tf.variable_scope("metrics"):
        metrics = {
            'accuracy': tf.metrics.accuracy(labels=tf.argmax(y,-1), predictions=tf.argmax(y_conv,-1)),
            'loss': tf.metrics.mean(cross_entropy)
        }
    
    # Group the update ops for the tf.metrics
    update_metrics_op = tf.group(*[op for _, op in metrics.values()])

    # Get the op to reset the local variables used in tf.metrics
    metric_variables = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metrics")
    metrics_init_op = tf.variables_initializer(metric_variables)
    
    
    # -----------------------------------------------------------
    # MODEL SPECIFICATION
    # Create the model specification and return it
    # It contains nodes or operations in the graph that will be used for training and evaluation
    variable_init_op = tf.group(*[tf.global_variables_initializer()])
    model_spec['variable_init_op'] = variable_init_op
    model_spec["predictions"] = y_conv
    model_spec['loss'] = cross_entropy
    model_spec['accuracy'] = accuracy
    model_spec['metrics_init_op'] = metrics_init_op
    model_spec['metrics'] = metrics
    model_spec['update_metrics'] = update_metrics_op
    model_spec['summary_op'] = tf.summary.merge_all()

    if is_training:
        model_spec['train_op'] = train_step
    
    return model_spec

In [19]:
def train_sess(sess, model_spec, num_steps, writer, params):
    """Train the model on `num_steps` batches

    Args:
        sess: (tf.Session) current session
        model_spec: (dict) contains the graph operations or nodes needed for training
        num_steps: (int) train for this number of batches
        writer: (tf.summary.FileWriter) writer for summaries
        params: (Params) hyperparameters
    """
    # Get relevant graph operations or nodes needed for training
    loss = model_spec['loss']
    train_op = model_spec['train_op']
    update_metrics = model_spec['update_metrics']
    metrics = model_spec['metrics']
    summary_op = model_spec['summary_op']
    y_conv = model_spec["predictions"]
    global_step = tf.train.get_global_step()

    # Load the training dataset into the pipeline and initialize the metrics local variables
    sess.run(model_spec['iterator_init_op'])
    sess.run(model_spec['metrics_init_op'])

    # Use tqdm for progress bar
    t = trange(num_steps)
    for i in t:
        # Evaluate summaries for tensorboard only once in a while
        if i % params["save_summary_steps"] == 0:
            # Perform a mini-batch update
            _, _, loss_val, summ, global_step_val = sess.run([train_op, update_metrics, loss,summary_op, global_step])
            # Write summaries for tensorboard
            print("Global Step: ",global_step_val)
            writer.add_summary(summ, global_step_val)
        else:
            _, _, loss_val = sess.run([train_op, update_metrics, loss])
        # Log the loss in the tqdm progress bar
        t.set_postfix(loss='{:05.3f}'.format(loss_val))
        print("Predictions",sess.run([y_conv[0:10,]]))

    metrics_values = {k: v[0] for k, v in metrics.items()}
    metrics_val = sess.run(metrics_values)
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_val.items())
    logging.info("- Train metrics: " + metrics_string)

In [6]:
def evaluate_sess(sess, model_spec, num_steps, writer=None, params=None):
    """Train the model on `num_steps` batches.

    Args:
        sess: (tf.Session) current session
        model_spec: (dict) contains the graph operations or nodes needed for training
        num_steps: (int) train for this number of batches
        writer: (tf.summary.FileWriter) writer for summaries. Is None if we don't log anything
        params: (Params) hyperparameters
    """
    update_metrics = model_spec['update_metrics']
    eval_metrics = model_spec['metrics']
    global_step = tf.train.get_global_step()

    # Load the evaluation dataset into the pipeline and initialize the metrics init op
    sess.run(model_spec['iterator_init_op'])
    sess.run(model_spec['metrics_init_op'])

    # compute metrics over the dataset
    for _ in range(num_steps):
        sess.run(update_metrics)

    # Get the values of the metrics
    metrics_values = {k: v[0] for k, v in eval_metrics.items()}
    metrics_val = sess.run(metrics_values)
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_val.items())
    logging.info("- Eval metrics: " + metrics_string)

    # Add summaries manually to writer at global_step_val
    if writer is not None:
        global_step_val = sess.run(global_step)
        for tag, val in metrics_val.items():
            summ = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=val)])
            writer.add_summary(summ, global_step_val)

    return metrics_val

In [7]:
def evaluate(model_spec, model_dir, params, restore_from):
    import pandas as pd
    """Evaluate the model

    Args:
        model_spec: (dict) contains the graph operations or nodes needed for evaluation
        model_dir: (string) directory containing config, weights and log
        params: (Params) contains hyperparameters of the model.
                Must define: num_epochs, train_size, batch_size, eval_size, save_summary_steps
        restore_from: (string) directory or file containing weights to restore the graph
    """
    # Initialize tf.Saver
    saver = tf.train.Saver()
    
    df = pd.DataFrame()
    with tf.Session() as sess:
        # Initialize the lookup table
        #sess.run(model_spec['variable_init_op'])
        # Reload weights from the weights subdirectory
        save_path = os.path.join(model_dir, restore_from)
        if os.path.isdir(save_path):
            save_path = tf.train.latest_checkpoint(save_path)
        saver.restore(sess, save_path)
        
        sess.run(model_spec['iterator_init_op'])
        totalBatches = (104170//params["TEST_BATCH_SIZE"]) + 1
        
        for index in range(totalBatches):        
            predictions,query_id,passage_id = sess.run([model_spec["predictions"],model_spec["query_id"],model_spec["passage_id"]])
            
            tmp = pd.DataFrame({"query_id":query_id[:,0],"passage_id":passage_id[:,0],"predictions":predictions[:,1]})
            df = pd.concat([df,tmp],axis = 0)
            #print(query_id[0:100])
    
    return df

In [8]:
def train_and_evaluate(train_model_spec, eval_model_spec, model_dir, params):
    """Train the model and evaluate every epoch.

    Args:
        train_model_spec: (dict) contains the graph operations or nodes needed for training
        eval_model_spec: (dict) contains the graph operations or nodes needed for evaluation
        model_dir: (string) directory containing config, weights and log
        params: (Params) contains hyperparameters of the model.
                Must define: num_epochs, train_size, batch_size, eval_size, save_summary_steps
        restore_from: (string) directory or file containing weights to restore the graph
    """
    
    max_query_words = params["max_query_words"]
    max_passage_words = params["max_passage_words"]
    emb_dim = params["emb_dim"]
    BATCH_SIZE = params["BATCH_SIZE"]
    EPOCHS = params["EPOCHS"]
    BatchesPerEpoch = params["BatchesPerEpoch"]
    num_classes = params["num_classes"]
    
    # Initialize tf.Saver instances to save weights during training
    last_saver = tf.train.Saver() # will keep last 5 epochs
    best_saver = tf.train.Saver(max_to_keep=1)  # only keep 1 best checkpoint (best on eval)
    begin_at_epoch = 0
    
    with tf.Session() as sess:
        # Initialize model variables
        sess.run(train_model_spec['variable_init_op'])

        # For tensorboard (takes care of writing summaries to files)
        train_writer = tf.summary.FileWriter(os.path.join(model_dir, 'train_summaries'), sess.graph)
        eval_writer = tf.summary.FileWriter(os.path.join(model_dir, 'eval_summaries'), sess.graph)

        best_eval_acc = 0.0
        for epoch in range(EPOCHS):
            # Run one epoch
            logging.info("Epoch {}/{}".format(epoch + 1, EPOCHS))
            
            train_sess(sess, train_model_spec, BatchesPerEpoch, train_writer, params)

            # Save weights
            last_save_path = os.path.join(model_dir, 'last_weights', 'after-epoch')
            last_saver.save(sess, last_save_path, global_step=epoch + 1)

            metrics = evaluate_sess(sess, eval_model_spec, 1, eval_writer)

            # If best_eval, best_save_path
            eval_acc = metrics['accuracy']
            if eval_acc >= best_eval_acc:
                # Store new best accuracy
                best_eval_acc = eval_acc
                # Save weights
                best_save_path = os.path.join(model_dir, 'best_weights', 'after-epoch')
                best_save_path = best_saver.save(sess, best_save_path, global_step=epoch + 1)
                logging.info("- Found new best accuracy, saving in {}".format(best_save_path))
                # Save best eval metrics in a json file in the model directory
                best_json_path = os.path.join(model_dir, "metrics_eval_best_weights.json")
                save_dict_to_json(metrics, best_json_path)

            # Save latest eval metrics in a json file in the model directory
            last_json_path = os.path.join(model_dir, "metrics_eval_last_weights.json")
            save_dict_to_json(metrics, last_json_path)

In [20]:
#-----------------Main function for training

tf.reset_default_graph()
train_model_spec = model_fn("train","./trainEmbeddings.tfrecords",params)
eval_model_spec = model_fn("eval","./validationEmbeddings.tfrecords",params)
logging.info("Starting training for {} epoch(s)".format(params["EPOCHS"]))
train_and_evaluate(train_model_spec, eval_model_spec, "./ModelLogs", params)

Tensor("model/query/convLayer2/maxPool:0", shape=(?, 2, 8, 2), dtype=float32)
Tensor("model_1/query/convLayer2/maxPool:0", shape=(?, 2, 8, 2), dtype=float32)







  0%|                                                                                            | 0/1000 [00:00<?, ?it/s]

Global Step:  0







  0%|                                                                                | 0/1000 [00:00<?, ?it/s, loss=0.671]

Predictions [array([[0.15584895, 0.08165452],
       [0.15936944, 0.08062461],
       [0.14658819, 0.10190961],
       [0.14653747, 0.09895856],
       [0.16198692, 0.10846816],
       [0.1523965 , 0.10335493],
       [0.17963877, 0.12931609],
       [0.15435189, 0.08586035],
       [0.15851013, 0.08492937],
       [0.13694662, 0.0864806 ]], dtype=float32)]


...

Predictions [array([[0.33347157, 0.        ],
       [0.29243618, 0.        ],
       [0.2709846 , 0.        ],
       [0.39594   , 0.        ],
       [0.25125843, 0.01280931],
       [0.38402444, 0.        ],
       [0.29871237, 0.        ],
       [0.30515277, 0.01320011],
       [0.3537882 , 0.        ],
       [0.32623968, 0.        ]], dtype=float32)]







  8%|█████▎                                                                 | 75/1000 [00:19<04:08,  3.72it/s, loss=0.573]




  8%|█████▎                                                                 | 75/1000 [00:19<04:08,  3.72it/s, loss=0.575]

Predictions [array([[0.39641577, 0.        ],
       [0.37053317, 0.        ],
       [0.2721934 , 0.        ],
       [0.37044683, 0.        ],
       [0.2547342 , 0.02028164],
       [0.37386218, 0.        ],
       [0.33743104, 0.        ],
       [0.38231716, 0.        ],
       [0.35307115, 0.        ],
       [0.31094337, 0.        ]], dtype=float32)]







  8%|█████▍                                                                 | 76/1000 [00:19<04:06,  3.75it/s, loss=0.575]

KeyboardInterrupt: 

In [ ]:
#-----------------Main function for testing

tf.reset_default_graph()
eval_model_spec = modelTest_fn("test","./evalUnlabelledEmbeddings.tfrecords",params)
logging.info("Starting training for {} epoch(s)".format(params["EPOCHS"]))
df = evaluate(eval_model_spec, "./ModelLogs", params,"best_weights")

import csv
tmp = (df.groupby('query_id')['predictions']
       .apply(lambda x: "\t".join([format(val, "0.2f") for val in x]))
       .reset_index())

tmp.to_csv("./answer.tsv",index=False,sep= "\t",header=None,quoting=csv.QUOTE_NONE,quotechar="",  escapechar="\\")

In [ ]:
#df.head(100)
#map(str,x)
import csv
tmp = (df.groupby('query_id')['predictions']
       .apply(lambda x: "\t".join([format(val, "0.2f") for val in x]))
       .reset_index())

tmp.to_csv("./answer.tsv",index=False,sep= "\t",header=None,quoting=csv.QUOTE_NONE,quotechar="",  escapechar="\\")

In [ ]:
df[df["query_id"] == 89]["predictions"].apply(lambda x: "\t".join(format(x, "0.2f")))

In [ ]:
tmp.head()